### 1. Подготовка

Для подготовки к сбору данных из [Co.op Online](https://cooponline.vn/) мы импортируем необходимые библиотеки для веб-скрейпинга и работы с данными. Переменная `CHECK_DATE` хранит текущую дату, которая используется для маркировки файлов.

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import random
import pandas as pd
import json
from datetime import date

CHECK_DATE = date.today()

### 2. Веб-скрейпинг

Чтобы получить данные о товарах с Co.op, мы действуем поэтапно: сначала получаем категории товаров, затем коды товаров с соответствующих страниц, собираем информацию о каждом товаре по категориям, очищаем полученные данные и сохраняем их в структурированном формате.

#### 2.1. Получение категорий

Начинаем с извлечения категорий товаров из сохранённой HTML-страницы (`groups.html`), она как и любая другая страница сайта Co.op Online содержит структуру каталога товаров. Скрипт проходит по трём уровням дерева категорий: верхний уровень, подкатегории и подподкатегории.

Каждая категория сохраняется в виде словаря с полями: *name*, *level* (1 — верхний уровень, 2 — подкатегория, 3 — подподкатегория), флаг наличия дочерних категорий, родительская категория (если есть) и *link*, ссылка на страницу категории.

In [ ]:
with open("groups.html", "r", encoding="utf-8") as file:
    groups_page = BeautifulSoup(file, "html.parser")    # загружаем сохранённую HTML-страницу и создаём объект BeautifulSoup

categories = []
# находим все категории верхнего уровня
top_categories = groups_page.find_all("a", class_="clearfix", href=re.compile(r"https://cooponline.vn/groups/[^#]"))

# проходим по всем категориям и извлекаем нужные данные
for category in top_categories:
    
    categories.append({        # сохраняем только нужные поля в словарь
        "name": category.span.string, # имя категории для справки
        "level": 1, # уровень категории
        "hasChild": True, # задаём True, потому что у верхних категорий всегда есть подкатегории, это важно для выгрузки товаров
        "parent": None, # родительская категория отсутствует
        "link": category["href"] # ссылка на страницу категории, нужна для получения кодов товаров
    })
    
    submenu = category.find_next_sibling("div", class_="sub-menu")    # подкатегории находятся в последующем блоке div с классом sub-menu
    subcategories = submenu.find_all("a", class_="main-menu")    # находим все подкатегории и проходим по ним
    for subcategory in subcategories:

        subsubmenu = subcategory.find_next_sibling("ul")    # проверяем, есть ли у подкатегории свои дочерние категории
        
        categories.append({    # так же добавляем подкатегории в список словарей
            "name": subcategory.string,
            "level": 2,
            "hasChild": True if subsubmenu else False,   # если есть подподкатегории, ставим True, иначе False
            "parent": category.span.string,
            "link": subcategory["href"]
        })

        # если есть подподкатегории, то проходим и по ним
        if subsubmenu: 
            
            subsubcategories = subsubmenu.find_all("a")
            for subsubcategory in subsubcategories:
                
                categories.append({
                    "name": subsubcategory.string,
                    "level": 3,
                    "hasChild": False,
                    "parent": subcategory.string,
                    "link": subsubcategory["href"]
                })

После формирования списка мы приводим названия категорий к единому виду, удаляем пробелы в начале и в конце.

In [ ]:
for category in categories:
    category["name"] = category["name"].strip()
    if category["parent"] is not None:
        category["parent"] = category["parent"].strip()

#### 2.2. Получение кодов товаров

В отличие от других сайтов в этом проекте, Co.op Online требует явного указания кодов товаров в теле POST-запроса при получении данных о них. Эти коды встроены в HTML-код страницы каждой категории. Чтобы перейти к выгрузке товаров, необходимо сначала перейти на страницу каждой категории и извлечь соответствующие коды.

Хотя коды товаров указаны для категорий всех уровней, мы сфокусируемся на тех, у которых нет подкатегорий. Эти категории меньше по объёму, что поможет снизить риск блокировки или потери большого количества данных при ошибке запроса. На каждой странице категории мы находим тег `module-taxonomy`, который содержит `term_id` категории и список кодов товаров в атрибуте `items`. Сохраняем эту информацию вместе с метаданными категории для следующего этапа.

In [ ]:
# создаём файл с заголовком таблицы для записи данных
categories_df = pd.DataFrame(columns=["name", "level", "hasChild", "parent", "link", "term_id", "item_codes"])
categories_df.to_csv(f'categories-{CHECK_DATE}-coop.csv', index=False, mode='w')

total_categories_count = len(categories) # общее количество категорий, которые нужно обработать (для отслеживания прогресса)
fetched_categories_count = 0 # счётчик обработанных категорий

for category in categories:
    
    print(f'{category["name"]}..', end='') # выводим текущую категорию
    
    if not category["hasChild"]: # если у категории нет подкатегорий, обрабатываем её
        
        print('нет подкатегорий, ищем коды товаров', end='') # выводим, что идёт поиск кодов товаров

        # загружаем HTML-страницу категории, извлекаем term_id и item_codes и сохраняем их
        current_page = requests.get(category["link"]).text
        page_bs = BeautifulSoup(current_page, "html.parser")
        products_tag = page_bs.find("module-taxonomy")
        if products_tag is not None:
            category["term_id"] = products_tag["term_id"]
            category["item_codes"] = products_tag["items"]

        # сохраняем категорию в DataFrame и добавляем строку в CSV-файл
        categories_df = pd.DataFrame([category])
        categories_df.to_csv(f'categories-{CHECK_DATE}-coop.csv', index=False, mode='a', header=False)
    else: # если у категории есть подкатегории, HTML не загружаем, но сохраняем информацию в CSV
        print('есть подкатегории, пропускаем', end='')
        category["term_id"] = None
        category["item_codes"] = None
        categories_df = pd.DataFrame([category])
        categories_df.to_csv(f'categories-{CHECK_DATE}-coop.csv', index=False, mode='a', header=False)
        
    fetched_categories_count += 1
    print(f'..обработано - {fetched_categories_count} из {total_categories_count}')
    
    time.sleep(random.uniform(1, 3))

Загружаем итоговую таблицу с категориями и заменяем отсутствующие значения на `None`, чтобы избежать ошибок на следующих этапах.

In [ ]:
categories = pd.read_csv('categories-2025-03-06-coop-complete.csv', dtype={"term_id": str})
categories = categories.where(pd.notna(categories), None)  # преобразуем NaN в None
categories = categories.to_dict('records')
# categories = categories[:] # эту строку можно использовать, чтобы продолжить сбор данных с определённого места в случае ошибки

#### 2.3. Получение исходных данных о товарах

Чтобы получить список товаров с Co.op Online, необходимо воспроизвести поведение сайта при просмотре пользователем страницы категории. В отличие от сайтов, где данные о товарах запрашиваются только на основе кода или slug категории, Co.op требует, чтобы в каждом POST-запросе передавался конкретный список кодов товаров.

Мы перебираем каждую категорию без подкатегорий и используем ранее извлечённые `term_id` и коды товаров, чтобы отправить POST-запросы на сервер сайта. Каждый запрос возвращает до 24 товаров. Постраничная загрузка реализуется через параметр `trang`, который увеличивается с каждой итерацией. Запросы продолжаются до тех пор, пока количество возвращённых товаров не станет меньше 24 — это значит, что список закончился.

Чтобы избежать блокировки или ограничения частоты запросов, между запросами добавляется случайная задержка. В результате получаем список словарей с «сырыми» данными о товарах в формате JSON.

In [ ]:
ITEMS_HEADERS = {
    'origin': 'https://cooponline.vn',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
}

In [ ]:
def fetch_products(category):
    
    current_products = 24 # задаем начальное количество товаров равным 24, чтобы гарантировать хотя бы одну итерацию цикла
    current_page = 1
    products = []

    # задаем заголовок referer в соответствии с текущей категорией
    ITEMS_HEADERS['referer'] = category["link"]
    url = 'https://cooponline.vn/ajax/'
        
    while current_products >= 24: # продолжаем цикл, пока на текущей странице не меньше 24 товаров
        
        time.sleep(random.uniform(1, 5)) # случайная задержка, чтобы избежать блокировки или ограничений

        # составляем тело POST-запроса
        DATA = {
            'request': 'w_getProductsTaxonomy',
            'termid': category["term_id"],
            'taxonomy': 'groups',
            'store': 'xtanphong',
            'items': category["item_codes"],
            'trang': current_page,
        }

        # получаем данные о товарах для текущей категории и страницы
        response = requests.post('https://cooponline.vn/ajax/', headers=ITEMS_HEADERS, data=DATA)
        response_data = json.loads(response.text) # преобразуем JSON-ответ в список словарей
        products += response_data # добавляем товары в список
        
        current_products = len(response_data) # обновляем количество загруженных товаров
        current_page += 1 # переходим к следующей странице
        
        print(f'{len(products)}..', end='') # индикатор прогресса

    return products 

#### 2.4. Очистка данных о товарах

После сбора товаров извлекаем только основную информацию для анализа. А именно:

- Название категории (для контекста)
- Название товара
- Цена
- Единица измерения
- Название супермаркета

«Сырые» данные из формата JSON конвертируем в более простую и понятную структуру. В результате имеем список словарей, каждый из которых представляет упрощённую запись о товаре.

In [ ]:
def clean_product_data(category, raw_products):

    cleaned_products = []

    # проходим по сырым данным и выбираем только нужные поля
    for product in raw_products:
        cleaned_products.append({
            "category_name": category["name"],
            "name": product["name"],
            "price": product["price"],
            "uom": product["unit"],
            "supermarket": "Co.op"
        })

    return cleaned_products

#### 2.5. Сводим все вместе

В следующей ячейке кода:

1. Проходим по выбранным категориям, содержащим коды товаров,
2. Получаем и очищаем данные по товарам каждой категории,
3. Добавляем очищенные данные в общий CSV-файл, собирая полный набор данных.

Для удобства отслеживания в процессе выводится количество загруженных товаров по каждой категории. Между итерациями добавляется небольшая случайная задержка, чтобы избежать ограничения по количеству запросов. Имя выходного файла содержит дату (`CHECK_DATE`), чтобы зафиксировать, когда данные были выгружены.

In [ ]:
fetched_categories_count = 0    # счётчик категорий для отслеживания прогресса загрузки
total_categories_count = len(categories)

# создаём файл с заголовком таблицы для записи данных
products_df = pd.DataFrame(columns=["category_name", "name", "price", "uom", "supermarket"])
products_df.to_csv(f'scraped_products-{CHECK_DATE}-coop.csv', index=False, mode='w')

for category in categories:
    
    if category["item_codes"] is not None: # проверяем, что категория не пустая
        
        raw_products = fetch_products(category) # загружаем товары
        new_products = clean_product_data(category, raw_products) # извлекаем только необходимые данные и добавляем новые товары в список
    
        products_df = pd.DataFrame(new_products)
        products_df.to_csv(f'scraped_products-{CHECK_DATE}-coop.csv', index=False, mode='a', header=False)

        print(f'Категория "{category["name"]}" обработана..', end='')
    
    else:
        print(f'Пропускаем категорию "{category["name"]}"..', end='')    
    
    fetched_categories_count += 1
    print(f'Загружено {fetched_categories_count} из {total_categories_count} категорий')
    
    time.sleep(random.uniform(1, 5))

print(f'Загрузка завершена. Результаты сохранены в файл scraped_products-{CHECK_DATE}-coop.csv')

### 3. Фильтрация и нормализация данных о товарах

После сбора и очистки исходных данных о продуктах мы переходим к фильтрации, чтобы включить только те товары, которые мы планируем сравнивать. Этот этап состоит из нескольких шагов:

#### 3.1. Первичная обработка

Начинаем с загрузки ранее сохранённых данных о товарах и категориях.

- Удаляем поле `category_name`, так как оно не нужно далее.
- Удаляем повторяющиеся записи о товарах — они могли появиться, если один и тот же товар был в нескольких категориях.
- Удаляем лишние пробелы в названиях товаров.

In [ ]:
import pandas as pd
import re

categories = pd.read_csv('categories-2025-03-06-coop-complete.csv')
products_original = pd.read_csv('scraped_products-2025-03-07-coop-complete.csv')

products = products_original.drop(['category_name'], axis=1)    # удаляем поле category_name
products = products.drop_duplicates() # удаляем дубли
products.loc[:,'name'] = products['name'].str.strip() # удаляем лишние пробелы

#### 3.2. Фильтрация товаров по типу

Чтобы выбрать из набора только релевантные для сравнения товары, мы создаём словарь с типами продуктов и соответствующими регулярными выражениями. Каждое выражение выбирает базовую форму продукта, исключая вариации, которые не входят в сферу нашего анализа.

> **Примечание:** приведенные здесь регулярные выражения не универсальны и подобраны под конкретный набор данных.

In [ ]:
product_regex_map = {
    'rice': r'^gạo(?!.*(lứt|lức|dưỡng|nếp))',
    'bread': r'^bánh (mì|mỳ|sandw|bag)(?!.*(bông|thịt|bơ|kem|hoa cúc|gà|pate|xốt|sữa|floss|socola|khoai|trứng|trong|hươu|nho|smile))',
    'chicken_fillet': r'(file|phi lê|\bức)(?!.*đùi).*gà',
    'pork_leg': r'đùi.*heo',
    'egg': r'^trứng gà(?!.*(ăn liền|tiềm|nướng|cay))', # выбирает "trứng gà", но исключает "trứng vịt" и готовые к употреблению яйца
    'cucumber': r'^dưa.*leo',
    'carrot': r'^cà rốt',
    'onion': r'hành tây',
    'tomato': r'^cà chua(?!.*(puree|đặc))',
    'cabbage': r'bắp cải trắng',
    'banana': r'^chuối(?!.*sấy)',
    'orange': r'^cam\b(?!.*sấy)',
    'milk': r'^sữa (tươi|tiệt|dinh|vina)(?!.*(melon|chuối|trái cây|có đường|ít đường|soco|dâu|vani|trân châu|ngữ|choco|lacto))',
    'yogurt': r'^sữa chua(?!.*(uống|men|khô|dẻo|ml))',
    'condensed_milk': r'sữa đặc(?!.*xanh lá)',
    'black_tea': (
        r'^(hồng trà|trà\b)'
        r'(?!.*(ml|l\b|xanh|sữa|khổ|sen|atiso|hoa cúc|ô long|olong|o long|green|ice|nestea|thảo|gừng|lài'
        r'|matcha|chia|sâm|thế hệ|hà thủ|thái nguyên|15g|blendy|linh chi|happy|tân cương|huế|tết|thanh nhiệt))'),
    'green_tea': (
        r'^trà\b'
        r'(?!.*(ml|l\b|sữa|khổ|atiso|hoa cúc|ice|nestea|thảo|gừng|matcha|chia|thế hệ|hà thủ|blendy|linh chi|happy'
        r'|huế|thanh nhiệt|dilmah|twinings|tết|tim sen|chanh|tâm sen|đen|lipton|dâu|bạc hà|hàn quốc|đào|quất))'),
    'ground_coffee': r'^(cà phê|cafe)(?!.*(hòa tan|hoà tan|sữa|in1|nesca|hạt|425g|bịch|fin|cino|hương))',
    # ↑ выбирает молотый кофе, но исключает растворимый или кофе с добавками
    'sugar': r'^đường\s(tinh|trắng|mía|kính)',
    'salt': r'^muối(?!.*(tôm|ớt|tiêu)).*(biển|iot|tinh|sạch)',
    'sunflower_oil': r'^dầu.*hướng dương',
    'soybean_oil': r'dầu.*nành',
    'water': r'nước\s(uống đóng|khoáng|tinh)(?!.*(ion|chanh|perr))',
    'spaghetti': r'^mì(?!.*(kool|trộn|bò|omto|kem)).*(ý|spag|hair|buca)',
    'rice_noodles': r'^(bún|phở)(?!.*(lứt|đen|60g|65g|\sg$)).*(wai|minh hảo|nuffam|bình tây|sa đéc|saf|select|mikiri|hùng lô)',
    'tofu': r'^(đậu|tàu)\shũ(?!.*(chiên|trứng|cá\b|nấm|hạt|ky))', # выбирает свежий тофу, исключает жареный и с добавками
    'water_spinach': r'^rau.*muống',
    'mango': r'^xoài(?!.*(sấy|ngâm))',
    'fish_sauce': r'^nước mắm(?!.*(ớt|me\b|gừng|chua\b|chay|tỏi|ngừ|nục|ăn liền))'
}
product_regex_list = '|'.join(product_regex_map.values()) # объединяем все регулярные выражения в одно, используя оператор OR (|)

# фильтруем товары, названия которых соответствуют одному из выбранных типов продуктов
filtered_products = products.loc[products.name.str.contains(product_regex_list, case=False, regex=True)]

#### 3.3. Присвоение типа продукта

Каждому отфильтрованному товару присваивается метка с типом продукта на основе совпадения по регулярному выражению.

In [ ]:
def assign_product_type(row):
    name = row['name']
    for product_type, regex in product_regex_map.items():
        match = re.search(regex, name, flags=re.IGNORECASE)
        if match:
            return product_type
    return None

filtered_products = filtered_products.copy()  # создаём копию, чтобы избежать предупреждений при добавлении новых полей через .loc
filtered_products.loc[:,'product_type'] = filtered_products.apply(assign_product_type, axis=1)

# необязательный промежуточный вывод списка отфильтрованных товаров
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.4. Извлечение и нормализация единиц измерения

Разные единицы товаров отличаются по количеству, весу или объёмы. Чтобы сравнение было корректным, мы извлекаем нужную информацию из названия товара или единицы измерения и рассчитываем нормализованные показатели — цену за килограмм, литр или за штуку.

- Вес в граммах
- Количество штук (в частности, яиц)
- Объём в миллилитрах

Каждое из значений извлекается по шаблону (через регулярное выражение). Не все товары содержат все эти значения, поэтому соответствующие колонки с нормализованными ценами (*price_kg*, *price_lit*, *price_unit*) могут остаться пустыми для некоторых товаров.

In [ ]:
def extract_weight(row):
    """Извлекает общий вес в граммах из названия товара или единицы измерения.
    Поддерживает как одиночные товары, так и формат нескольких упаковок в одном (например, '5x100g', '5 gói x 100g').
    Если упомянуты только килограммы (явно или косвенно), по умолчанию устанавливает 1 кг.
    """
    
    name, uom = row['name'], row['uom']
    
    # формат с несколькими порциями (первым указан вес)
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(g\b|gr\b)\s?(x|gói)\s?(\d+)', name, flags=re.IGNORECASE) # matches digits g х digits
    if match:
        portion = int(match.group(4)) # извлекаем количество порций
        per_portion = float(match.group(1).replace(',', '.')) # извлекаем вес порции
        return portion * per_portion # возвращаем общий вес
    # формат с несколькими порциями (вес указан вторым)  
    match = re.search(r'(\d+)(\s|\shủ\s?|\shộp\s?|\sgói\s?|\stúi\s?)?x\s?(\d+|\d+[,.]\d+)\s?g\b', name, flags=re.IGNORECASE)
    if match:
        portion = float(match.group(1).replace(',', '.'))
        per_portion = float(match.group(3).replace(',', '.'))
        return portion * per_portion
    # одиночный вес (в граммах или килограммах)
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(g\b|gr\b|kg)', name, flags=re.IGNORECASE)
    if match:
        weight = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return weight * 1000 if unit in ['kg','Kg'] else weight # преобразуем килограммы в граммы, если необходимо
    # если в названии ничего нет, проверяем единицу измерения (uom)
    if uom == 'kg':
        weight = 1000
        return weight
    # если ничего из вышеуказанного не сработало, но в названии встречается 'kg'
    match = re.search(r'kg', name, flags=re.IGNORECASE)
    if match:
        weight = 1000
        return weight
    
    return None  # если ничего не найдено

# следующие две функции работают по той же логике, что и extract_weight, но для штук и миллилитров
def extract_number_of_units(row):
    """Извлекает количество единиц товара из названия или uom.
    Поддерживает форматы типа '10x', '10 túi', '10 trứng'.
    """
    
    name, product_type, uom = row['name'], row['product_type'], row['uom']
    
    # проверяем название
    match = re.search(r'(\d+)\s?(túi|gói|trứng|t\b|x)', name, flags=re.IGNORECASE) # túi - пакет, gói - упаковка, trứng/t - яйцо
    if match:
        number_of_units = int(match.group(1))
        return number_of_units
   
def extract_volume(row):
    """Извлекает общий объём в миллилитрах из названия товара.
    Поддерживает как одиночные товары, так и формат нескольких упаковок в одном (например, '5x100ml', 'thùng 6 x 330ml').
    """
    
    name, uom = row['name'], row['uom']
    
    # формат с несколькими порциями (первым указан объём)
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(ml|l\b|lít)\s?(x|thùng)\s?(\d+)', name, flags=re.IGNORECASE) # thùng - коробка
    if match:
        portion = int(match.group(4))
        per_portion = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return portion * per_portion * 1000 if unit in ['l', 'L', 'lít'] else portion * per_portion
    # формат с несколькими порциями (объём указан вторым)    
    match = re.search(r'(\d+)(\s|\sgói\s?|\sbịch\s?|\shộp\s?|\schai\s?)?[x×]\s?(\d+|\d+[,.]\d+)\s?(ml|l\b|lít)',
                      name, flags=re.IGNORECASE) # bịch - пакет, hộp - коробка, chai - бутылка
    if match:
        portion = int(match.group(1))
        per_portion = float(match.group(3).replace(',', '.'))
        unit = match.group(4)
        return portion * per_portion * 1000 if unit in ['l', 'L', 'lít'] else portion * per_portion
    # одиночный объём (в литрах или миллилитрах)
    match = re.search(r'(\d+|\d+[,.]\d+)\s?(ml|l\b|lít)', name, flags=re.IGNORECASE)
    if match:
        volume = float(match.group(1).replace(',', '.'))
        unit = match.group(2)
        return volume * 1000 if unit in ['l', 'L', 'lít'] else volume
    
    return None  # если ничего не найдено

filtered_products = filtered_products.copy()  # пересоздаем датафрейм

# вычисляем нормализованные цены
filtered_products.loc[:,'weight'] = filtered_products.apply(extract_weight, axis=1)  # столбец с весом в граммах
filtered_products.loc[:,'price_kg'] = filtered_products.price / filtered_products.weight * 1000   # столбец с ценой за кг

filtered_products.loc[:,'number_of_units'] = filtered_products.apply(extract_number_of_units, axis=1)  # столбец с количеством штук
filtered_products.loc[:,'price_unit'] = filtered_products.price / filtered_products.number_of_units   # столбец с ценой за штуку

filtered_products.loc[:,'volume'] = filtered_products.apply(extract_volume, axis=1)  # столбец с объемом в мл
filtered_products.loc[:,'price_lit'] = filtered_products.price / filtered_products.volume * 1000   # столбец с ценой за литр

# необязательный промежуточный вывод
# with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
#     display(filtered_products)

#### 3.5. Сохранение финального отфильтрованного набора данных

Наконец, отфильтрованный и дополненный набор данных сохраняется в новый CSV-файл для дальнейшего анализа.

In [ ]:
filtered_products.to_csv(f'filtered_products-2025-03-07-coop.csv')